In [ ]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import json

Loading scores generated by the NGCF Model for every user-song pair

In [ ]:
triplets = pd.read_csv('normalised_triplets.csv')
users = sorted(list(set(triplets['user'])))
songs = sorted(list(set(triplets['song'])))
user_inverse_mapping = {}
song_inverse_mapping = {}
for i in range(len(users)):
    user_inverse_mapping[str(i)] = users[i]

for i in range(len(songs)):
    song_inverse_mapping[str(i)] = songs[i]

In [ ]:
user_num = [str(i) for i in range(13624)]

In [ ]:
ngcf_recs_np = np.load('rec_scores.npy', allow_pickle=True) #loading the scores file
ngcf_recs_np.shape

Filtering top 100 recommendation

In [ ]:
ngcf_recs = {}
for i in tqdm(range(ngcf_recs_np.shape[0]), ncols=100):
    ngcf_recs[user_inverse_mapping[user_num[i]]] = sorted([(song_inverse_mapping[str(j)], 
                                                            float(ngcf_recs_np[i][j])) for j in range(len(list(ngcf_recs_np[i])))], 
                                                          key=lambda x: x[1], reverse=True)[:100]

In [ ]:
with open('ngcf_emb20_only_user.json','w') as f:
    json.dump(ngcf_recs,f)

Creating a list of top 10 recommended songs for every user

In [ ]:
path = "/content/drive/MyDrive/Recommendations/"
with open("/content/drive/MyDrive/NGCF_Float64/NGCF/ngcf_emb20_only_user.json", "r") as fp:
    final_recommendation_dic = json.load(fp)
ngcf_dic = {}
count = 0
k = 10
for i in tqdm(final_recommendation_dic,ncols=100):
    ngcf_dic[count] = {'user':i,'ngcf_predictions':[j[0] for j in final_recommendation_dic[i][:10]]}
    # ngcf_dic[count] = {'user':i,'ngcf_embedding_predictions':[j[0] for j in final_recommendation_dic[i][:10]]}
    count+=1
ngcf_recs_df = pd.DataFrame.from_dict(ngcf_dic, orient='index')
ngcf_recs_df.to_csv(path+"ngcf_emb20_only_user.csv", index=False)
ngcf_recs_df.shape